In [ ]:
!pip install facenet-pytorch
!pip install expecttest
import io
import os
import zipfile
import cv2
from PIL import Image
from torchvision import transforms
from facenet_pytorch import MTCNN
import torch

# Initialize MTCNN face detector with GPU support
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
mtcnn = MTCNN(keep_all=False, device=device)  # Set device to GPU if available

# Define the transformation to resize to 224x224
resize_transform = transforms.Resize((224, 224))

# Function to process each folder
def process_folder(input_folder, output_zip):
    processed_files = 0  # Track number of processed files
    
    # Create the zip file for output
    with zipfile.ZipFile(output_zip, 'w', zipfile.ZIP_DEFLATED) as zipf:
        # Define the padding for bounding box (to ensure enough margin around the face)
        padding = 10  # Adjust as needed

        # Iterate over all images in the input folder
        for subdir, dirs, files in os.walk(input_folder):
            for file in files:
                image_path = os.path.join(subdir, file)

                if file.lower().endswith(('.jpg', '.jpeg', '.png')):  # Process image files only
                    # Read and process each image
                    frame = cv2.imread(image_path)
                    
                    if frame is not None:
                        # Convert frame directly to PIL Image (as it's already RGB)
                        frame_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)  # Ensure correct color format
                        frame_image = Image.fromarray(frame_rgb)
                        
                        # Detect face(s) in the image
                        faces, probs = mtcnn.detect(frame_image)  # MTCNN returns a list of faces and their probabilities

                        if faces is not None and len(faces) > 0:
                            # Get the bounding box of the first detected face
                            x1, y1, x2, y2 = faces[0]
                            
                            # Add padding to the bounding box
                            x1 = max(0, int(x1) - padding)
                            y1 = max(0, int(y1) - padding)
                            x2 = min(frame_rgb.shape[1], int(x2) + padding)
                            y2 = min(frame_rgb.shape[0], int(y2) + padding)
                            
                            # Crop the face region from the original image with padding
                            cropped_face = frame_rgb[y1:y2, x1:x2]
                            cropped_face_image = Image.fromarray(cropped_face)
                            
                            # Resize the cropped face to 224x224
                            resized_face = resize_transform(cropped_face_image)
                            
                            # Save the resized face into the zip as a properly encoded JPEG
                            with io.BytesIO() as buffer:
                                resized_face.save(buffer, format="JPEG")
                                zipf.writestr(f"face_{os.path.basename(image_path)}", buffer.getvalue())
                            processed_files += 1  # Increment the processed file counter
                        
        return processed_files

# Define the output zip files dictionary for each method
output_zip_files = {
    output/folder/location
}

# List of input folders corresponding to each method 
input_folders = {
    input/folder/location # Replace with actual folder
}

# Process each method and create separate zip files for each
for method, input_folder in input_folders.items():
    output_zip = output_zip_files[method]
    processed_files = process_folder(input_folder, output_zip)
    print(f"Processed {processed_files} files from {method} method. Extracted faces are saved in {output_zip}")

print("All methods have been processed. Zip files with extracted faces are created.")